In [1]:
import pandas as pd
import re
from Bio import SeqIO

In [18]:
a = list(SeqIO.parse("D45mix.genbank", 'genbank'))

In [19]:
b = a[0]

In [20]:
c = b.features[3]

In [23]:
c.qualifiers['locus_tag'][0]

'OVNPSFZQ_3'

In [250]:
fin_anno = "rstBhNR.m8"
colnames = ['query', 'target', 'evalue', 'gapopen', 'pident', 'fident', 'nident', 'qstart', 'qend', 'qlen', 'tstart', 'tend', 'tlen', 'bits', 'qheader', 'theader', 'mismatch', 'qcov', 'tcov', 'taxid', 'taxname', 'taxlineage']
df = pd.read_csv(fin_anno, sep='\t', names=colnames)

In [251]:
df#!/usr/bin/env python

import click
import pandas as pd
import re

def pretty_theader(theader):
    theader = theader.split("\x01")
    annotations = [re.sub("\[.*", "", x) for x in theader]
    anno1 = re.sub("^[.\w]+\s", "", annotations[0], 1)
    rst_anno = [re.sub("^.*: ", "", anno1).strip()]
    if len(annotations) > 1:
        anno_opt = [re.sub("^[.\w]+\s", "", x, 1).strip() for x in annotations[1:]]
        anno_opt = [x for x in anno_opt if rst_anno[0] not in x]
        anno_opt = list(set(anno_opt))
        rst_anno += anno_opt

    # choose functions that are not assigned as hypothetical protein
    rst_anno = [x for x in rst_anno if "hypothetical protein" not in x]

    return rst_anno


@click.command()
@click.option("--fin", '-i', help="input file name")
@click.option("--fout", '-o', help="output file name")
def main(fin, fout, excel):
    df = pd.read_csv("test.tsv", sep="\t")
    df['annotation'] = df.apply(lambda x: pretty_theader(x['theader']), axis=1)
    df['function'] = df.apply(lambda x: '' if len(x['annotation'])==0 else x['annotation'][0], axis=1)
    df['function_opt'] = df.apply(lambda x: '' if len(x['annotation'])<=1 else '\n'.join(x['annotation'][1:]), axis=1)

    # select columns
    cols = ['query', 'target', 'pident', 'bits', 'evalue', 'taxid', 'function', 'function_opt']
    df = df[cols]

    # save to file
    df.to_csv(fout, sep="\t", index=False)


if __name__ == '__main__':
    main()

,query,target,evalue,gapopen,pident,fident,nident,qstart,qend,qlen,...,tlen,bits,qheader,theader,mismatch,qcov,tcov,taxid,taxname,taxlineage
0,D45s01_15_3899,WP_063972546.1,3.451000e-37,0,64.4,0.644,0,2,114,116,...,116,146,D45s01_15_3899 25129_25479,WP_063972546.1 MULTISPECIES: hypothetical prot...,40,0.974,0.974,37914,Dietzia,-_cellular organisms;d_Bacteria;-_Terrabacteri...
1,D45s01_16_3995,WP_107757771.1,5.766000e-310,0,96.9,0.969,0,1,466,466,...,466,953,D45s01_16_3995 38890_40290,WP_107757771.1 MULTISPECIES: LLM class flavin-...,14,1.000,1.000,2617939,unclassified Dietzia,-_cellular organisms;d_Bacteria;-_Terrabacteri...
2,D45s01_17_4027,WP_107756329.1,3.553000e-223,0,92.2,0.922,0,1,360,360,...,360,696,D45s01_17_4027 9479_10561,WP_107756329.1 MULTISPECIES: dephospho-CoA kin...,28,1.000,1.000,2617939,unclassified Dietzia,-_cellular organisms;d_Bacteria;-_Terrabacteri...
3,D45s04_1_452,WP_107756648.1,9.858000e-92,0,88.5,0.885,0,1,165,165,...,165,306,D45s04_1_452 380298_379801,WP_107756648.1 MULTISPECIES: 30S ribosomal pro...,19,1.000,1.000,2617939,unclassified Dietzia,-_cellular organisms;d_Bacteria;-_Terrabacteri...
4,D45s04_1_516,WP_231750271.1,8.030000e-162,0,93.2,0.932,0,243,511,511,...,269,527,D45s04_1_516 447237_445702,WP_231750271.1 MULTISPECIES: DoxX family prote...,18,0.526,1.000,2617939,unclassified Dietzia,-_cellular organisms;d_Bacteria;-_Terrabacteri...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91401,DMPs06_22_4230,WP_107755579.1,2.149000e-283,0,79.9,0.799,0,1,454,454,...,642,876,DMPs06_22_4230 14332_15693,WP_107755579.1 MULTISPECIES: tetratricopeptide...,91,1.000,0.832,2617939,unclassified Dietzia,-_cellular organisms;d_Bacteria;-_Terrabacteri...
91402,DMPs09_5_4028,WP_119849456.1,9.971000e-257,0,99.0,0.990,0,1,379,379,...,379,794,DMPs09_5_4028 450800_449661,WP_119849456.1 tRNA guanosine(34) transglycosy...,4,1.000,1.000,2,Bacteria,-_cellular organisms;d_Bacteria
91403,DMPs09_6_4092,MCA4756313.1,6.417000e-154,0,99.4,0.994,0,1,232,232,...,232,489,DMPs09_6_4092 33874_33176,MCA4756313.1 HNH endonuclease [Mycolicibacteri...,1,1.000,1.000,1766,Mycolicibacterium fortuitum,-_cellular organisms;d_Bacteria;-_Terrabacteri...
91404,DMPs09_6_4156,WP_127590950.1,3.755000e-128,0,100.0,1.000,0,1,187,193,...,486,413,DMPs09_6_4156 79032_78451,WP_127590950.1 sulfatase [Paenibacillus lautus],0,0.969,0.385,1401,Paenibacillus lautus,-_cellular organisms;d_Bacteria;-_Terrabacteri...


In [223]:
fin_gbk = "z.gbk"
fout = "new.gbk"
fin_anno = "rst.tsv"
col_feature_id = "query"
key_feature_id = "locus_tag"
col_func = "function"
key_func = "anno_nr"
key_product = "product"
col_func_opt = "function_opt"

In [240]:
from dataclasses import replace


anno_db = key_func
df_anno = pd.read_csv(fin_anno, sep='\t', index_col='query')
df_anno.rename(columns={"function": "function2"}, inplace=True)
anno = df_anno.to_dict(orient='index')

In [113]:
# ID mapping of FASTA
rcs_fasta = list(SeqIO.parse("phage28.faa", "fasta"))
id_mapping_faa = pd.DataFrame([x.description.split(' ') for x in rcs_fasta], columns=['prot_id', 'loc_range'])
id_mapping_faa['genome_id'] = id_mapping_faa.apply(lambda x: '_'.join(x['prot_id'].split('_')[:-1]), axis=1)
id_mapping_faa['feature_id'] = id_mapping_faa.apply(lambda x: '_'.join([x['genome_id'], x['loc_range']]), axis=1)

id_mapping_faa

,prot_id,loc_range,genome_id,feature_id
0,D45mix_1_0,1_267,D45mix_1,D45mix_1_1_267
1,D45mix_1_1,306_791,D45mix_1,D45mix_1_306_791
2,D45mix_1_2,788_1039,D45mix_1,D45mix_1_788_1039
3,D45mix_1_3,1036_1287,D45mix_1,D45mix_1_1036_1287
4,D45mix_1_4,1287_1559,D45mix_1,D45mix_1_1287_1559
...,...,...,...,...
124273,DMPs11_44_4348,732_1,DMPs11_44,DMPs11_44_732_1
124274,DMPs11_45_4349,1_150,DMPs11_45,DMPs11_45_1_150
124275,DMPs11_45_4350,126_458,DMPs11_45,DMPs11_45_126_458
124276,DMPs11_46_4351,158_3,DMPs11_46,DMPs11_46_158_3


In [243]:
for gr in rcs_gbk:
    for feature in gr.features:
        if feature.type == "CDS":
            af = anno.get(feature.qualifiers['feature_id'][0]) # annotation dict of feature
            if af:
                feature.qualifiers.update(af)


In [145]:
id_mapping = pd.merge(id_mapping_gbk, id_mapping_faa, on='feature_id', how='left')

In [146]:
id_mapping

,genome_id_x,locus_tag,location_range,feature_id,prot_id,loc_range,genome_id_y
0,D45mix_1,OLJIMSTB_0,1_267,D45mix_1_1_267,D45mix_1_0,1_267,D45mix_1
1,D45mix_1,OLJIMSTB_1,306_791,D45mix_1_306_791,D45mix_1_1,306_791,D45mix_1
2,D45mix_1,OLJIMSTB_2,788_1039,D45mix_1_788_1039,D45mix_1_2,788_1039,D45mix_1
3,D45mix_1,OLJIMSTB_3,1036_1287,D45mix_1_1036_1287,D45mix_1_3,1036_1287,D45mix_1
4,D45mix_1,OLJIMSTB_4,1287_1559,D45mix_1_1287_1559,D45mix_1_4,1287_1559,D45mix_1
...,...,...,...,...,...,...,...
1258,D45mix_3,OLJIMSTB_416,33491_32592,D45mix_3_33491_32592,D45mix_3_416,33491_32592,D45mix_3
1259,D45mix_3,OLJIMSTB_417,35588_33501,D45mix_3_35588_33501,D45mix_3_417,35588_33501,D45mix_3
1260,D45mix_3,OLJIMSTB_418,35863_35552,D45mix_3_35863_35552,D45mix_3_418,35863_35552,D45mix_3
1261,D45mix_3,OLJIMSTB_419,35865_35954,D45mix_3_35865_35954,D45mix_3_419,35865_35954,D45mix_3


In [125]:
feature

SeqFeature(FeatureLocation(ExactPosition(35969), ExactPosition(36563), strand=1, ref='D45mix_3'), type='CDS', ref='D45mix_3')

In [147]:
id_mapping_gbk.to_csv("a.tsv")
id_mapping_faa.to_csv("faa.tsv")
id_mapping.to_csv("zzz.tsv")

In [159]:
import os
from BCBio import GFF
from Bio.Seq import Seq

In [ ]:
#!/usr/bin/env python

import click
from BCBio import GFF
from Bio.Seq import Seq


@click.command()
@click.option("--fin_gff", '-f', help="input gff file")
@click.option("--fin_fasta", '-a', help="input gbk file")
@click.option("--fout_gbk", '-o', help="output gbk file")
def main(fin_gff, fin_fasta, fout_gbk):
    with open(fout_gbk, "wt") as gbk_handler:
        fasta_handler = SeqIO.to_dict(SeqIO.parse(fin_fasta, "fasta"))
        for record in GFF.parse(fin_gff, fasta_handler):
            for feature in record.features:
                locus_tag = feature.qualifiers['locus_tag'][0]
                genome_prokka_id = locus_tag.split('_')[0]
                feature_id = locus_tag.replace(genome_prokka_id, record.id)
                feature.qualifiers.update({'feature_id': feature_id})
                feature.qualifiers.update({'translation': Seq.translate(record.seq[feature.location.start.position:feature.location.end.position], to_stop=True)})
            record.annotations["molecule_type"] = "DNA"
            SeqIO.write(record, gbk_handler, "genbank")


if __name__ == '__main__':
    main()

In [191]:
rcs_faa = SeqIO.to_dict(SeqIO.parse("phanotate_aas.fasta", "fasta"))

In [195]:
str(rcs_faa['D45mix_1_0'].seq)

'ARDEETGESVILKAKDGVFEYGEGVDERNIKRLQDAGLSKVLGMLSKQTDWSKWVDQEVWRAPAGYDRGLLSKEWIESQPAEVNVLTR'

In [190]:
for f in record.features:
    if f.strand!=1:
        print(f.strand)

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


In [171]:
# GFF -> GBK
fin_gff = "D45mix.gff"
fout_gbk = "abc.gbk"
fin_faa = "D45mix_scaffolds.fasta"

with open(fout_gbk, "wt") as gbk_handler:
    fasta_handler = SeqIO.to_dict(SeqIO.parse(fin_faa, "fasta"))
    for record in GFF.parse(fin_gff, fasta_handler):
        for feature in record.features:
            feature.qualifiers.update({'translation': Seq.translate(record.seq[feature.location.start.position:feature.location.end.position], to_stop=True)})
        record.annotations["molecule_type"] = "DNA"
        SeqIO.write(record, gbk_handler, "genbank")

In [166]:
fasta_handler = SeqIO.to_dict(SeqIO.parse("phanotate_aas.fasta", "fasta"))
rcs_gff = list(GFF.parse("D45mix.gff"))

In [21]:
gr = records[0]
# gr.features
for feature in gr.features[:10]:
    if feature.type == "CDS":
        feature_id = feature.qualifiers.get(key_feature_id)
        if feature_id:
            feature_id = feature_id[0]
        df2 = df_anno[df_anno[col_feature_id]==feature_id]

,genome_id,locus_tag,location_range
0,D45mix_1,OLJIMSTB_0,1_267
1,D45mix_1,OLJIMSTB_1,306_791
2,D45mix_1,OLJIMSTB_2,788_1039
3,D45mix_1,OLJIMSTB_3,1036_1287
4,D45mix_1,OLJIMSTB_4,1287_1559
...,...,...,...
1294,D45mix_3,OLJIMSTB_416,32592_33491
1295,D45mix_3,OLJIMSTB_417,33501_35588
1296,D45mix_3,OLJIMSTB_418,35552_35863
1297,D45mix_3,OLJIMSTB_419,35865_35954


In [ ]:
for genome_region in records:
    genome_region = update_others(genome_region, df, anno_db)

In [6]:
for feature in genome_region.features:
    if feature.type == "CDS":
        locus_tag = feature.qualifiers.get('locus_tag')
        if locus_tag:
            locus_tag = locus_tag[0]
        df2 = df[df.protid==locus_tag]
        if df2.shape[0]>0:
            feature.qualifiers[anno_db] = list(df2.anno.unique())
    return genome_region

,query,target,pident,bits,evalue,taxid,function,function_opt
0,D45s01_15_3899,WP_063972546.1,64.4,146,3.451000e-37,37914,NaN,NaN
1,D45s01_16_3995,WP_107757771.1,96.9,953,5.766000e-310,2617939,LLM class flavin-dependent oxidoreductase,"5,10-methylene tetrahydromethanopterin reducta..."
2,D45s01_17_4027,WP_107756329.1,92.2,696,3.553000e-223,2617939,dephospho-CoA kinase,NaN
3,D45s04_1_452,WP_107756648.1,88.5,306,9.858000e-92,2617939,30S ribosomal protein S16,NaN
4,D45s04_1_516,WP_231750271.1,93.2,527,8.030000e-162,2617939,DoxX family protein,NaN
...,...,...,...,...,...,...,...,...
91401,DMPs06_22_4230,WP_107755579.1,79.9,876,2.149000e-283,2617939,tetratricopeptide repeat protein,TPR repeat protein
91402,DMPs09_5_4028,WP_119849456.1,99.0,794,9.971000e-257,2,tRNA guanosine(34) transglycosylase Tgt,queuine tRNA-ribosyltransferase
91403,DMPs09_6_4092,MCA4756313.1,99.4,489,6.417000e-154,1766,HNH endonuclease,NaN
91404,DMPs09_6_4156,WP_127590950.1,100.0,413,3.755000e-128,1401,sulfatase,NaN


In [ ]:
#!/usr/bin/env python

import click
from Bio import SeqIO
import pandas as pd


@click.command()
@click.option("--fin_anno", '-i', help="input file name")
@click.option("--fin_gbk", '-g', help="input file name")
@click.option("--fout_gbk", '-o', help="input file name")
@click.option("--col_feature_id", '-f', default="query", help="input file name")
@click.option("--key_feature_id", '-F', default="feature_id", help="input file name")
@click.option("--col_anno", '-a', default="function", help="input file name")
@click.option("--key_anno", '-A', default="anno_nr", help="input file name")
def main(fin_anno, fin_gbk, fout_gbk, col_feature_id, key_feature_id, col_anno, key_anno):
    rcs_gbk = list(SeqIO.parse(fin_gbk, format="genbank"))
    df_anno = pd.read_csv(fin_anno, sep='\t', index_col=col_feature_id)
    df_anno.rename(columns={col_anno: key_anno}, inplace=True)
    df_anno = df_anno[[key_func]]
    anno = df_anno.to_dict(orient='index')

    for gr in rcs_gbk:
        for feature in gr.features:
            if feature.type == "CDS":
                af = anno.get(feature.qualifiers[key_feature_id][0]) # annotation dict of feature
                if af:
                    feature.qualifiers.update(af)


    with open(fout_gbk, "w") as fh:
        SeqIO.write(rcs_gbk, fh, "genbank")


if __name__ == '__main__':
    main()